In [50]:
import collections
import itertools

In [51]:
sos = '<s>'
eos = '</s>'
def read_docs(dataset_year: str, max_docs_count: int):
  is_doc = False
  docs_count = 0

  with open(f'dataset/cg.Folha.{dataset_year}', encoding='windows-1252') as file: 
    for line in file:
      if docs_count >= max_docs_count:
        break

      if line.startswith('<DOC>'):
        is_doc = True
        continue

      if line.startswith('</DOC>'):
        is_doc = False
        docs_count += 1
        continue

      if is_doc is False:
        continue

      if line.startswith(sos):
        yield (sos, sos)
      elif line.startswith(eos):
        yield (eos, eos)
      else:
        splitted_line = line.split('\t')
        if len(splitted_line) == 2:
          word = splitted_line[0]
          tag = splitted_line[1].split()[1]
          yield (word, tag)

In [52]:
word_tag_counter = collections.Counter()
tag_counter = collections.Counter()

for word, tag in read_docs('1994', max_docs_count=1):
  tag_counter.update({ tag: 1 })
  word_tag_counter.update({ (word, tag): 1 })

tag_counter



Counter({'<s>': 4,
         'PROP': 20,
         'V': 7,
         'PRP': 21,
         'DET': 10,
         'NUM': 7,
         'ADJ': 7,
         'N': 14,
         'PU': 21,
         'ADV': 2,
         '</s>': 4,
         'KC': 2,
         '<prop>': 1})